In [28]:
import networkx as nx
import numpy as np

In [29]:
def read_community_labels_file_reel(file_path):
    """
    Reads the ground truth files for the reel datasets.

    Args:
        file_path (str): The path to the file containing the community labels.

    Returns:
        list: A list of tuples representing the node and its corresponding label.
              Each tuple contains the node index (int) and the label (int+1).
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()
        res = []
        for node, label in enumerate(lines):
            res.append((int(node), int(label)+1))
        return res

In [30]:
def similarity_matrix(A: np.ndarray) -> np.ndarray:
    """
    This function takes an adjacency matrix A and returns the similarity matrix S.
    """
    # Calculate the Similarity Matrix S
    S = np.dot(A, A.T)
    return S

In [31]:
def distance_matrix(similarity_matrix):
    """
    Converts an unnormalized similarity matrix to a distance matrix.
    This function first normalizes the similarity scores based on the maximum value found in the matrix.

    Parameters:
    similarity_matrix (numpy.ndarray): A square matrix containing unnormalized similarity scores.

    Returns:
    numpy.ndarray: A square matrix containing distance scores.
    """
    # Validate the input matrix is square
    if similarity_matrix.shape[0] != similarity_matrix.shape[1]:
        raise ValueError("The similarity matrix must be square.")

    # Normalize similarity scores by the maximum score in the matrix
    max_similarity = np.max(similarity_matrix, axis=1)
    normalized_similarity = similarity_matrix / max_similarity


    # Convert normalized similarity to distance
    distance_matrix = 1 - normalized_similarity

    return distance_matrix

In [32]:
def fitness_function(adj_matrix: np.ndarray, nodes: list, alpha=.9, beta=1.1) -> float:
    """
    This function calculates the fitness function for a given subgraph defined by the nodes.
    It returns the number of edges inside the subgraph divided by the sum of edges inside and coming out of the subgraph.
    """

    subgraph = adj_matrix[nodes][:, nodes]

    num_edges_inside = np.count_nonzero(np.triu(subgraph))

    unused_nodes = [node for node in list(
        range(adj_matrix.shape[0])) if node not in nodes]

    num_edges_outside = np.count_nonzero(adj_matrix[nodes][:, unused_nodes])

    return num_edges_inside / (num_edges_inside**alpha + num_edges_outside**beta)

In [33]:
def average_weight(dist_matrix: np.ndarray, nodes: list) -> float:
    """
    This function takes a distance matrix D and a list of nodes and returns the average weight of the nodes.
    specific for complete graphs
    """

    filtered_dist_matrix = dist_matrix[nodes][:, nodes]
    sum_distances = np.sum(filtered_dist_matrix)  # sum of distances
    nb_edges = len(nodes) * (len(nodes) - 1)   # cause it's a complete graph

    return sum_distances / nb_edges

In [34]:
def estimate_number_k(G: nx.Graph, alpha=.9, beta=1.1,min_per_clique= 3):
    """
    This function takes a graph G and returns a list of k initial seeds.
    """


    A = nx.to_numpy_array(G)
    S = similarity_matrix(A)

    # Calculate the distance matrix D using S
    D = distance_matrix(S)
    adj_matrix = nx.to_numpy_array(G)

    # find all complete subgraphs of size 3 or more in the graph
    cliques = find_cliques(G, D)

    # sort the cliques by their weight
    cliques_sorted = sorted(cliques, key=lambda x: x["weight"], reverse=True)

    # get the unselected nodes
    unselected_nodes = set(G.nodes())
    skip_nodes = []

    M = 0
    while  cliques_sorted:

        # get the node with the maximum degree
        max_degree_node = max(unselected_nodes, key=lambda node:  G.degree(
            node) * int(node not in skip_nodes))

        chosen_clique = None
        chosen_clique_index = -1
        # get the clique that contains the node with the maximum degree
        for i, clique in enumerate(cliques_sorted):
            if max_degree_node in clique["nodes"]:
                chosen_clique_index = i
                chosen_clique = clique["nodes"]
                break

        # if the node with the maximum degree is not in any clique we skip it in next iteration
        if not chosen_clique:
            skip_nodes.append(max_degree_node)
            continue

        cliques_sorted.pop(chosen_clique_index)

        # sort the nodes in the clique by their distance to the other nodes in the clique
        condidat_nodes_in_order = sorted(
            unselected_nodes, key=lambda node: min(D[node, target_node] for target_node in chosen_clique))

        # get the initial fitness value of the chosen clique
        fitness_value = fitness_function(adj_matrix, chosen_clique)

        # add the nodes that maximizes the fitness function to the chosen clique
        for node in condidat_nodes_in_order:
            if node not in chosen_clique:
                fitness = fitness_function(
                    adj_matrix, chosen_clique + [node], alpha, beta)

                if fitness >= fitness_value:
                    fitness_value = fitness
                    chosen_clique.append(node)



        # remove the nodes in the chosen clique from the unselected nodes
        unselected_nodes.difference_update(chosen_clique)

        # remove the treeted nodes from the cliques list
        new_cliques = []
        for clique in cliques_sorted:
            clique["nodes"] = [node for node in clique["nodes"]
                               if node in unselected_nodes]
            clique["weight"] = average_weight(adj_matrix, clique["nodes"])

            if len(clique["nodes"]) >= min_per_clique:
                new_cliques.append(clique)

        # sort the new cliques according to their weight
        cliques_sorted = sorted(
            new_cliques, key=lambda x: x["weight"], reverse=True)

        M += 1
    return M

In [35]:

def find_cliques(G: nx.Graph, adj_matrix: np.ndarray) -> list:
    """
    This function takes a graph G and returns a list of all cliques in the graph.
    """

    cliques_generator = nx.find_cliques(G)

    res = []
    for clique in cliques_generator:
        if len(clique) > 2:
            res.append({
                "nodes": clique,
                "weight": average_weight(adj_matrix, clique)
            })

    return res

In [36]:
ALPHA = .93
BETA = 1.07

In [37]:

file_path = '../data/reel/karate/karate.gml'
G = nx.read_gml(file_path, label='id')
original_nodes = list(G.nodes())
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)
adj_matrix = nx.to_numpy_array(G)
true_labels = read_community_labels_file_reel(
    '../data/reel/karate/groundTruth.txt')
print(f"The graph contains {adj_matrix.shape[0]} nodes.")

The graph contains 34 nodes.


In [38]:
estimate_number_k(G, alpha=ALPHA, beta=BETA)

C:\Users\ASUS ROG\AppData\Local\Temp\ipykernel_36292\1566038148.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return sum_distances / nb_edges


2

In [39]:

file_path = '../data/reel/dolphins/dolphins.gml'
G = nx.read_gml(file_path, label='id')
original_nodes = list(G.nodes())
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)
adj_matrix = nx.to_numpy_array(G)
true_labels = read_community_labels_file_reel(
    '../data/reel/dolphins/groundTruth.txt')
print(f"The graph contains {adj_matrix.shape[0]} nodes.")

The graph contains 62 nodes.


In [40]:
estimate_number_k(G, alpha=ALPHA, beta=BETA)

C:\Users\ASUS ROG\AppData\Local\Temp\ipykernel_36292\1566038148.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return sum_distances / nb_edges


3

In [41]:

file_path = '../data/reel/football/football.gml'
G = nx.read_gml(file_path, label='id')
original_nodes = list(G.nodes())
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)
adj_matrix = nx.to_numpy_array(G)
true_labels = read_community_labels_file_reel(
    '../data/reel/football/groundTruth.txt')
print(f"The graph contains {adj_matrix.shape[0]} nodes.")

The graph contains 115 nodes.


In [42]:
estimate_number_k(G, alpha=ALPHA, beta=BETA)

C:\Users\ASUS ROG\AppData\Local\Temp\ipykernel_36292\1566038148.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return sum_distances / nb_edges


11

In [43]:

file_path = '../data/reel/polbooks/polbooks.gml'
G = nx.read_gml(file_path, label='id')
original_nodes = list(G.nodes())
mapping = {node: i for i, node in enumerate(G.nodes())}
G = nx.relabel_nodes(G, mapping)
adj_matrix = nx.to_numpy_array(G)
true_labels = read_community_labels_file_reel(
    '../data/reel/polbooks/groundTruth.txt')
print(f"The graph contains {adj_matrix.shape[0]} nodes.")

The graph contains 105 nodes.


In [44]:
estimate_number_k(G, alpha=ALPHA, beta=BETA)

C:\Users\ASUS ROG\AppData\Local\Temp\ipykernel_36292\1566038148.py:11: RuntimeWarning: invalid value encountered in scalar divide
  return sum_distances / nb_edges


4

___